In [ ]:
import pandas as pd
import numpy as np
import jax.numpy as jnp
import numpyro
import matplotlib.pyplot as plt

from PacTimeOrig.data import DataHandling as dh
from PacTimeOrig.data import DataProcessing as dp
from PacTimeOrig.controllers import simulator as sim
from PacTimeOrig.controllers import JaxMod as jm
from PacTimeOrig.controllers import models as conmod
from PacTimeOrig.controllers import utils as ut
from PacTimeOrig.data import scripts


cfgparams={'area':'pMD','rbfs':[20,30,40,50,60,70,80],'restarts':6}

Compare pvif on glonbal then adam v just adam

In [ ]:
sess=0
cfgparams={'area':'dACC','session':sess+1,'subj':'H','typeswitch':1,'rbfs':[20,30,40,50,60,70,80],'restarts':6}

Xdsgn, kinematics, sessvars, psth = scripts.monkey_run(cfgparams)

trial=69
num_rbfs = 30


fitdat=ut.get_data_for_fit(Xdsgn, trial)

#Make time
tmp=ut.make_timeline(fitdat)

xin=np.hstack((fitdat['player_pos'],fitdat['player_vel']))
A,B=ut.define_system_parameters(ctrltype='pvif')


inputs=ut.prepare_inputs(A, B, xin, fitdat['uout'], fitdat['pry1_pos'], fitdat['pry2_pos'], tmp, num_rbfs, xin[:,2:], fitdat['pry1_vel'], fitdat['pry2_vel'], pry_1_accel=fitdat['pry1_accel'],pry_2_accel=fitdat['pry2_accel'],ctrltype='pvif',usingJax=True)




params = jm.initialize_parameters(inputs,ctrltype='pvif')

loss_function = jm.create_loss_function_pvif(ut.generate_rbf_basis,inputs['num_rbfs'],ut.generate_smoothing_penalty,lambda_reg=0.01)

grad_loss = jm.compute_loss_gradient(loss_function)



# Set up the optimizer
optimizer, opt_state = jm.setup_optimizer(params, learning_rate=1e-2,optimizer='amsgrad')

# Number of optimization steps
num_steps = 20000
# Optimization loop
for step in range(num_steps):
    params, opt_state, loss = jm.optimization_step(params, opt_state, optimizer, loss_function, inputs,slack_model=False)

    if step % 100 == 0:
        print(f"Step {step}, Loss: {loss}")


Ls_fit=jnp.log(1+jnp.exp(params[1][1:].reshape(2,4)))


weights=params[0]
widths=params[1][0]
w1,w2=conmod.generate_sim_switch(inputs, widths, weights)

outpred_g_a=sim.controller_sim_pvif_post(np.vstack((w1,w2)), fitdat['player_pos'], fitdat['player_vel'], fitdat['pry1_pos'], fitdat['pry1_vel'], fitdat['pry1_accel'], fitdat['pry2_pos'], fitdat['pry2_vel'],
                             fitdat['pry2_accel'], Ls_fit[0,:], Ls_fit[1,:], A=A, B=B, dt=1.0 / 60.0, Ie_min=-10.0, Ie_max=10.0)




In [ ]:
sess=0
cfgparams={'area':'dACC','session':sess+1,'subj':'H','typeswitch':1,'rbfs':[20,30,40,50,60,70,80],'restarts':6}

Xdsgn, kinematics, sessvars, psth = scripts.monkey_run(cfgparams)

trial=4
num_rbfs = 30


fitdat=ut.get_data_for_fit(Xdsgn, trial)
xin=np.hstack((fitdat['player_pos'],fitdat['player_vel']))

#Make time
tmp=ut.make_timeline(fitdat)

A,B=ut.define_system_parameters(ctrltype='pvif')

inputs=ut.prepare_inputs(A, B, xin, fitdat['uout'], fitdat['pry1_pos'], fitdat['pry2_pos'], tmp, num_rbfs, xin[:,2:], fitdat['pry1_vel'], fitdat['pry2_vel'], pry_1_accel=fitdat['pry1_accel'],pry_2_accel=fitdat['pry2_accel'],ctrltype='pvif',usingJax=True)



loss_function = conmod.create_loss_function_pvif(conmod.generate_rbf_basis)


grad_loss = conmod.compute_loss_gradient(loss_function)

(L1_opt, L2_opt), best_params, best_loss = conmod.outer_optimization_pvif(inputs, conmod.inner_optimization_pvif, loss_function, grad_loss,maxiter=5,tolerance=1e-3,opttype='global')


params=(jnp.array(best_params[0]),jnp.array(best_params[1]),jnp.array(L1_opt),jnp.array(L2_opt))

loss_function = jm.create_loss_function_pvif(ut.generate_rbf_basis,inputs['num_rbfs'])

grad_loss = jm.compute_loss_gradient(loss_function)

# Set up the optimizer
optimizer, opt_state = jm.setup_optimizer(params, learning_rate=1e-2)

# Number of optimization steps
num_steps = 30000
# Optimization loop
for step in range(num_steps):
    params, opt_state, loss = jm.optimization_step(params, opt_state, optimizer, loss_function, inputs)

    if step % 100 == 0:
        print(f"Step {step}, Loss: {loss}")


Ls_fit=jnp.log(1+jnp.exp(params[1][1:].reshape(2,4)))

weights=best_params[0:-1][0]
widths=best_params[-1]
w1,w2=conmod.generate_sim_switch(inputs, widths, weights)

outpred_g_a=sim.controller_sim_pvif_post(np.vstack((w1,w2)), fitdat['player_pos'], fitdat['player_vel'], fitdat['pry1_pos'], fitdat['pry1_vel'], fitdat['pry1_accel'], fitdat['pry2_pos'], fitdat['pry2_vel'],
                             fitdat['pry2_accel'], Ls_fit[0,:], Ls_fit[1,:], A=A, B=B, dt=1.0 / 60.0, Ie_min=-10.0, Ie_max=10.0)


lbfgs all